In [1]:
import pybamm
from ocv_teaching import OCV, ActiveMaterial, OCVBlending, CellOCVReconstruction

# Load parameterized OCV functions
param = pybamm.ParameterValues("Chen2020")
ocv_anode = param["Negative electrode OCP [V]"]
ocv_cathode = param["Positive electrode OCP [V]"]

cell = CellOCVReconstruction(
    cath_ocv_func=ocv_cathode,
    an_ocv_func=ocv_anode,
    np_ratio=1.1,
    v_min=2.5,
    v_max=4.2,
)

# Get stoichiometries
an0, cath0, an1, cath1 = cell.get_stoichiometries()

# Get voltage curve
volt_cell, volt_cath, volt_an = cell.reconstruct_voltage(an0, cath0, an1, cath1)



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Adjust path if the package is not installed
import sys
sys.path.append('.')  # Assumes root contains battery_ocv_toolbox/
# Import the package
from ocv_teaching import OCV, ActiveMaterial, OCVBlending, CellOCVReconstruction
from ocv_teaching.plot_ocv import plot_ocv
from ocv_teaching.utils import interpolate, new_plot

In [2]:
NCM811_df = pd.read_csv('../ocv_data/NMC811_half_cell_ocv.csv')
Graphite_df = pd.read_csv('../ocv_data/Graphite_half_cell_ocv.csv')

In [3]:
NCM811_ocv = OCV(NCM811_df["SOC"], NCM811_df["Voltage"], NCM811_df["Voltage"])
Graphite_ocv = OCV(Graphite_df["SOC"], Graphite_df["Voltage"], Graphite_df["Voltage"])
NCM811_mat = ActiveMaterial(NCM811_ocv, specific_capacity=212, formation_loss=0.09)
Graphite_mat = ActiveMaterial(Graphite_ocv, specific_capacity=372, formation_loss=0.08)


Plot OCVs vs capacity normalized

In [4]:

from ipywidgets import interact, FloatSlider
def plot_np_ratio(np_ratio,v_min,v_max):
    cell = CellOCVReconstruction(NCM811_mat, Graphite_mat, np_ratio=np_ratio, v_min=v_min, v_max=v_max)

    # Reconstruct voltage of the cell over the full range
    an0_full = cell.align_anode_cathode(1)
    an1_full = cell.align_anode_cathode(0)
    volt_cell_full  = cell.reconstruct_voltage(
        an0_full, 1, an1_full, 0, direction="discharge"
    )[0]
    # Get stoichiometeries within voltage limits
    an0, cath0, an1, cath1 = cell.get_stoichiometries()
    sol = np.linspace(0,1,100)
    # Relate anode to cathode stoichiometry
    sol_an_plot = (sol-an0_full)/(an1_full-an0_full)
    volt_cath = interpolate(cell.cath.ocv.soc, cell.cath.ocv.get_voltage("charge"), sol)
    volt_an = interpolate(cell.an.ocv.soc, cell.an.ocv.get_voltage("charge"), sol)

    # Plot the results
    battery_color={"gray": "#333333",
                       "bright_gray": "#E7E6E6",
                        "gray_blue": "#44546A",
                        "navy_blue": "#003B73",
                        "light_blue":"#5DADE2",
                        "orange": "#F39C12",
                        "green": "#70AD47"}
    fig, ax = new_plot(figsize=(8, 6))
    ax.set_xlim(0,1.2)
    ax.set_ylim(0,4.5)
    ax.set_ylabel("Voltage [V]")
    ax.set_xlabel("Normalized Capacity [Ah]")
    ax.grid()
    ax.plot(sol_an_plot , volt_an, label="Graphite OCV",color=battery_color["light_blue"])
    ax.plot(1-sol, volt_cath, label="NCM811 OCV",color=battery_color["green"])
    ax.plot(sol,volt_cell_full, label="Cell OCV", color=battery_color["navy_blue"])
    ax.hlines([v_min, v_max], 0, 1.2, linestyles='dashed', colors='black')
    ax.plot([1-cath0,1-cath1],[v_min,v_max],marker="o", color=battery_color["navy_blue"],linestyle='None')
    ax.legend(loc='right')

    # ## Draw the usage span
    # # compute the x‑coordinates of the usage span
    start = 1 - cath0
    end   = 1 - cath1
    mid   = 0.5*(start + end)
    usage = cath0 - cath1

    # draw a double‑headed arrow at y=3
    ax.annotate(
        '',                          # no text
        xy=(end, 3),                 # arrow head at right
        xytext=(start, 3),           # arrow tail at left
        arrowprops=dict(
            arrowstyle='<->',        # double‑headed
            color='black',
            lw=2
        )
    )

    # label it just above the arrow
    ax.text(
        mid, 3.05,                   # position (slightly above y=3)
        f'Cathode usage = {usage:.3f}',
        ha='center', va='bottom',
        color='black'
    )
    plt.show()

interact(
    plot_np_ratio,
    np_ratio=FloatSlider(min=0.9, max=1.2, step=0.01, value=1.1, description="N:P Ratio"),
    v_min=FloatSlider(min=2, max=3.6, step=0.01, value=2.8, description="Lower voltage limit"),
    v_max=FloatSlider(min=3.6, max=4.2, step=0.01, value=4.2, description="Upper voltage limit"),
)

interactive(children=(FloatSlider(value=1.1, description='N:P Ratio', max=1.2, min=0.9, step=0.01), FloatSlide…

<function __main__.plot_np_ratio(np_ratio, v_min, v_max)>